## Utilitaires

In [1]:
from copy import deepcopy
from numpy import sqrt
import re
import time

In [2]:
# Une image c'est 
# ses 4 côtés, 
# sa position dans la grille (si trouvée), 
# son orientation (mais ses côtés sont toujours données ESWN)
# ses différents voisins possibles qui dépendent de son orientation et de son flip
class Image():
    @classmethod
    def from_borders(cls, index, borders):
        side = len(borders[0])
        square = [''.join(borders[3])]
        for n in range(1, (side - 1)):
            square += [''.join([borders[2][n]] + ['o' * (side - 2)] + [borders[0][n]])]
        square += [''.join(borders[1])]
        return cls(index, square)
    
    def __init__(self, index, square):
        self.index = index
        self.square = square
        self.side = len(square)
        self.borders = [
            [row[-1] for row in square],  # East
            [x for x in square[-1]],      # South
            [row[0] for row in square],   # West
            [x for x in square[0]],       # North
        ]
        self.neighbours = []
        
    def neighboorhood(self, image):
        for n in range(4):
            fit_border = [
                p for p in range(4) 
                if ((self.borders[n] == image.borders[p]) or 
                    (self.borders[n][self.side::-1] == image.borders[p]))
            ]
            for p in range(len(fit_border)):
                self.neighbours = self.neighbours + [image]
    
    def draw(self):
        for row in range(self.side):
            print(self.square[row])
    
    def sub_square(self, coord, dim):
        return [row[coord[0]:(coord[0] + dim[0])] for row in self.square[coord[1]:(coord[1] + dim[1])]]
    
    def replace_sub_square(self, coord, sub_square):
        lx, ly = (len(sub_square[0]), len(sub_square))
        n = 0
        for y in range(coord[1], coord[1] + ly):
            self.square[y] = self.square[y][:coord[0]] + sub_square[n] + self.square[y][(coord[0] + lx):]
            n += 1
        self.borders = [
            [row[-1] for row in self.square],  # East
            [x for x in self.square[-1]],      # South
            [row[0] for row in self.square],   # West
            [x for x in self.square[0]],       # North
        ]
        return None

In [3]:
def orienter(image, orientation):
    flipv, fliph, rotation = orientation
    square = deepcopy(image.square)
    nrows, ncols = len(square), len(square[0])
    if flipv:
        for n in range(nrows):
            square[n] = square[n][::-1]
    if fliph:
        square = square[::-1]
    for turn in range(rotation):
        temp = [''.join([square[n - 1][p] for n in range(nrows, 0, -1)]) for p in range(ncols)]
        square = temp
        nrows, ncols = ncols, nrows
    
    return Image(image.index, square)

In [4]:
# Une grille c'est une nb de lignes et de colonnes 
# - les images correspondant à chaque coordonnée, avec leur orientation/flip
class Grid():
    def __init__(self, nrows, ncols):
        self.nrows = nrows
        self.ncols = ncols
        self.cells_borders = {(x, y): [None] * 4 for x in range(nrows) for y in range(ncols)}
        self.filled_cells = {(x, y): False for x in range(nrows) for y in range(ncols)}
        self.cells_images = {(x, y): None for x in range(nrows) for y in range(ncols)}
        self.cell_size = None
    
    def check_place_image(self, coords, image, orientation):
        return all([
            self.cells_borders[coords][n] in [orienter(image, orientation).borders[n], None] 
            for n in range(4)
        ])
    
    def place_image(self, coords, image, orientation):
        if (self.filled_cells[coords] == False) & self.check_place_image(coords, image, orientation):
            self.cells_images[coords] = image
            self.filled_cells[coords] = True
            oriented_image = orienter(image, orientation)
            self.cells_borders[(min(coords[0] + 1, self.ncols - 1), coords[1])][2] = oriented_image.borders[0]
            self.cells_borders[(max(0, coords[0] - 1), coords[1])][0] = oriented_image.borders[2]
            self.cells_borders[(coords[0], max(0, coords[1] - 1))][1] = oriented_image.borders[3]
            self.cells_borders[(coords[0], min(coords[1] + 1, self.nrows - 1))][3] = oriented_image.borders[1]
            self.cells_borders[coords] = oriented_image.borders
            if self.cell_size == None:
                self.cell_size = image.side
        else:
            print('Tuile non plaçable ici')
    
    def next_empty_cell(self):
        for coord in self.filled_cells:
            if ~self.filled_cells[coord]:
                return coord
        return None
    
    def full_square(self):
        if self.cell_size != None:
            c = self.cell_size
            full_square = ['~' * (c - 2) * self.ncols] * (c - 2) * self.nrows
            for coord, filled in self.filled_cells.items():
                if filled:
                    x, y = coord
                    r = 1
                    for row in range(y * (c - 2), (y + 1) * (c - 2)):
                        full_square[row] = (
                            full_square[row][:(x * (c - 2))] +
                            self.cells_images[coord].square[r][1:(c - 1)] +
                            full_square[row][((x + 1) * (c - 2)):]
                        )
                        r += 1
            return full_square
        return [f'Empty {self.nrows}x{self.ncols} grid']
    
    def draw(self):
        full_square = self.full_square()
        for row in range(len(full_square)):
            print(full_square[row])

In [5]:
def prepare_for_east(east, origin):
    nfit = east.borders.index(origin.borders[0])
    if nfit == 0:
        rotation = (True, False, 0)
    elif nfit == 1:
        rotation = (False, False, 1)
    elif nfit == 2:
        rotation = (False, False, 0)
    elif nfit == 3:
        rotation = (True, False, 3)
    return orienter(east, rotation)

In [6]:
def prepare_for_south(south, origin):
    nfit = south.borders.index(origin.borders[1])
    if nfit == 0:
        rotation = (False, False, 3)
    elif nfit == 1:
        rotation = (False, True, 0)
    elif nfit == 2:
        rotation = (False, True, 1)
    elif nfit == 3:
        rotation = (False, False, 0)
    return orienter(south, rotation)

In [7]:
def define_regex(monster):
    return [re.compile('^' + row.replace('#', '[0#]').replace('.', '[.~0#]') + '$') for row in monster]

In [8]:
def find_monster(rectangle, monster):
    monster_rules = define_regex(monster)
    ncols = len(monster[0])
    nrows = len(monster)
    
    if all([monster_rules[n].match(rectangle[n]) != None for n in range(nrows)]):
        rectangle = [
            ''.join([rectangle[y][x] if monster[y][x] == '.' else '0' for x in range(ncols)]) 
            for y in range(nrows)
        ]
        print('Monster found!')
    
    return rectangle

## Initialisations

In [9]:
start_time = time.time()

with open('input-files/input-day20.txt', 'r') as fic:
    tempo = fic.read().strip('\n').split('\n\n')

images = {}
for text in tempo:
    cut = text.split('\n')
    image_id = cut[0].split(' ')[1].strip(':')
    image = cut[1:]
    images[image_id] = image

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

Spent time: 0.001 s


## Run

### First star

In [10]:
start_time = time.time()

tiles = [Image(k, v) for k, v in images.items()]
ntiles = len(tiles)

dim = int(sqrt(ntiles))
jigsaw = Grid(dim, dim)

for tile in tiles:
    for neighbour in tiles:
        if neighbour != tile:
            tile.neighboorhood(neighbour)

corners = [tile.index for tile in tiles if len(tile.neighbours) == 2]

result = 1
for corner in corners:
    result *= int(corner)
print(result)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

16937516456219
Spent time: 0.170 s


### Second star

In [11]:
start_time = time.time()

corners = [tile for tile in tiles if len(tile.neighbours) == 2]
borders = [tile for tile in tiles if len(tile.neighbours) == 3]
inner = [tile for tile in tiles if len(tile.neighbours) == 4]

corner0 = corners[0]
neighbour0 = corner0.neighbours[0]
neighbour1 = corner0.neighbours[1]

# Placer le 1er voisin et tourner les 2 pièces pour pouvoir les mettre en position (0, 0) et (1, 0)
direct_fit = [n for n in range(4) if corner0.borders[n] in neighbour0.borders]
flipped_fit = [n for n in range(4) if corner0.borders[n][corner0.side::-1] in neighbour0.borders]

if len(direct_fit) > 0:
    nfit = direct_fit[0]
    if nfit == 0:
        rot0 = (False, False, 0)
    elif nfit == 1:
        rot0 = (True, False, 3)
    elif nfit == 2:
        rot0 = (True, False, 0)
    elif nfit == 3:
        rot0 = (False, False, 1)
elif len(flipped_fit) > 0:
    nfit = flipped_fit[0]
    if nfit == 0:
        rot0 = (False, True, 0)
    elif nfit == 1:
        rot0 = (True, False, 3)
    elif nfit == 2:
        rot0 = (True, True, 0)
    elif nfit == 3:
        rot0 = (True, False, 1)
im00 = orienter(corner0, rot0)

im10 = prepare_for_east(neighbour0, im00)

# 2ème voisin : au-dessus ou en dessous ? Si dessus, flip horizontal de corner et voisin1
if im00.borders[1][::-1] in neighbour1.borders:
    print('Il faut flipper verticalement coin et tuile 1 et les tourner de 3 quarts, tuile 2 à droite')
    im00 = orienter(im00, (True, False, 3))
    im01 = orienter(im10, (True, False, 3))
    im10 = prepare_for_east(neighbour1, im00)
    
else:
    print('Après réajustement, il faudra placer la tuile 2 en-dessous')
    if im00.borders[3] in neighbour1.borders:
        print('   Réajustement : Il faut flipper horizontalement le coin et la tuile 1')
        im00 = orienter(im00, (False, True, 0))
        im10 = orienter(im10, (False, True, 0))
    elif im00.borders[3][::-1] in neighbour1.borders:
        print('Il faut flipper verticalement coin et tuile 1 et les tourner de 1 demi')
        im00 = orienter(im00, (True, False, 2))
        im10 = orienter(im10, (True, False, 2))
    im01 = prepare_for_south(neighbour1, im00) 

jigsaw.place_image((0, 0), im00, (False, False, 0))
jigsaw.place_image((0, 1), im01, (False, False, 0))
jigsaw.place_image((1, 0), im10, (False, False, 0))

tiles.remove(corner0)
tiles.remove(neighbour0)
tiles.remove(neighbour1)

# De 2 à nbcols - 2: placer la tuile suivante, en la tournant si nécessaire (ligne 1)
# De 2 à nbrows - 2: placer la tuile suivante, en la tournant si nécessaire

for y in range(jigsaw.nrows):
    xmin = max(2 - y, 0)

    for x in range(xmin, jigsaw.ncols):
        if x == 0:
            north_tile = jigsaw.cells_images[(x, y - 1)]
            for tile in tiles:
                north_tile.neighboorhood(tile)
            
            if north_tile.borders[1] in north_tile.neighbours[0].borders:
                south_tile = prepare_for_south(north_tile.neighbours[0], north_tile)
                tiles.remove(north_tile.neighbours[0])
            elif north_tile.borders[1][::-1] in north_tile.neighbours[0].borders:
                south_tile = prepare_for_south(orienter(north_tile.neighbours[0], (True, True, 0)), north_tile)
                tiles.remove(north_tile.neighbours[0])
            elif north_tile.borders[1] in north_tile.neighbours[1].borders:
                south_tile = prepare_for_south(north_tile.neighbours[1], north_tile)
                tiles.remove(north_tile.neighbours[1])
            elif north_tile.borders[1][::-1] in north_tile.neighbours[1].borders:             
                south_tile = prepare_for_south(orienter(north_tile.neighbours[1], (True, True, 0)), north_tile)
                tiles.remove(north_tile.neighbours[1])
            
            jigsaw.place_image((x, y), south_tile, (False, False, 0))
        
        else:
            west_tile = jigsaw.cells_images[(x - 1, y)]
            for tile in tiles:
                west_tile.neighboorhood(tile)

            if west_tile.borders[0] in west_tile.neighbours[0].borders:
                east_tile = prepare_for_east(west_tile.neighbours[0], west_tile)
                tiles.remove(west_tile.neighbours[0])
            elif west_tile.borders[0][::-1] in west_tile.neighbours[0].borders:
                east_tile = prepare_for_east(orienter(west_tile.neighbours[0], (True, True, 0)), west_tile)
                tiles.remove(west_tile.neighbours[0])
            elif west_tile.borders[0] in west_tile.neighbours[1].borders:
                east_tile = prepare_for_east(west_tile.neighbours[1], west_tile)
                tiles.remove(west_tile.neighbours[1])
            elif west_tile.borders[0][::-1] in west_tile.neighbours[1].borders:
                east_tile = prepare_for_east(orienter(west_tile.neighbours[1], (True, True, 0)), west_tile)
                tiles.remove(west_tile.neighbours[1])

            jigsaw.place_image((x, y), east_tile, (False, False, 0))

# Trouver les monstres et les remplacer dans l'image dans des O
full_image = Image('result', jigsaw.full_square())

initial_hashtags = sum([sum([x == '#' for x in full_image.square[y]]) for y in range(full_image.side)])
print(initial_hashtags)

monster = ['..................#.', '#....##....##....###', '.#..#..#..#..#..#...']

for flipv in [False, True]:
    for fliph in [False, True]:
        for rotation in range(4):
            hidden_monster = orienter(Image('m', monster), (flipv, fliph, rotation)).square
            lx, ly = len(hidden_monster[0]), len(hidden_monster)
            for x in range(full_image.side - lx):
                for y in range(full_image.side - ly):
                    full_image.replace_sub_square(
                        (x, y), 
                        find_monster(full_image.sub_square((x, y), (lx, ly)), hidden_monster)
                    ) 

full_image.draw()

# Nombre de tous les # une fois les monstres placés
left_hashtags = sum([sum([x == '#' for x in full_image.square[y]]) for y in range(full_image.side)])
print(f'=== {left_hashtags} ===')

print(f'#monsters: {(initial_hashtags - left_hashtags) / 15:0.0f}')

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

Après réajustement, il faudra placer la tuile 2 en-dessous
2248
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
Monster found!
#..#............##....##.......#.....#...#......#.#...##.....###...#.......##......#....#..#..#.
.#..#........###..#....##..........#...#..#.......#..#..#..